In [1]:
%pylab inline
import pandas as pd
import seaborn
import numpy as np
import matplotlib.pyplot as plt
import scipy
import cv2
import nltk
import string
from collections import defaultdict
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
import sys
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest

Populating the interactive namespace from numpy and matplotlib


In [2]:
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

In [3]:
final_train_df = pd.read_json(r'C:\Users\HP\Documents\cmpt 459 Kaggle\two_sigma\final_train_df_m3_v2.json')
final_test = pd.read_json(r'C:\Users\HP\Documents\cmpt 459 Kaggle\two_sigma\final_test_df_m3_v2.json')

In [6]:
final_train_df["pos"] = final_train_df.longitude.round(3).astype(str) + '' + final_train_df.latitude.round(3).astype(str)
final_test["pos"] = final_test.longitude.round(3).astype(str) + '' + final_test.latitude.round(3).astype(str)

vals = final_train_df['pos'].value_counts()
dvals = vals.to_dict()
final_train_df["density"] = final_train_df['pos'].apply(lambda x: dvals.get(x, vals.min()))
final_test["density"] = final_test['pos'].apply(lambda x: dvals.get(x, vals.min()))

In [4]:
v2_selected_feats = ["bathrooms", "bedrooms", "latitude", "longitude", "price", "pet_policy", "listing_type", 
                     "unit_features", "building_features", "apartment", "appliance", "bathroom", "bedroom",
                     "building", "contact", "floor", "high", "kitchen", "room", "num_photos", 
                     "description_len", "mall_distance", "dt_distance", "managerID_count", "buildingID_count", 
                    "weekends", "month_4", "month_5", "created_nights","created_morning", "created_afternoon", 
                     "manager_skill", "building_interest", "density"]

In [7]:
X_series = final_train_df[v2_selected_feats]
y_series = final_train_df["interest_level"]

In [11]:
mi = mutual_info_classif(X_series, y_series, discrete_features=True)

In [48]:
feature_rank = {}
for i in range(len(v2_selected_feats)):
    if v2_selected_feats[i] not in feature_rank:
        feature_rank[v2_selected_feats[i]] = mi[i]
    print(v2_selected_feats[i], ' - ', mi[i])


bathrooms  -  0.00988099331734487
bedrooms  -  0.0061955250953194
latitude  -  0.08985502052948602
longitude  -  0.060959171000756256
price  -  0.08776029436638774
pet_policy  -  0.002376913208605575
listing_type  -  0.0007542008478153882
unit_features  -  0.008595750400336737
building_features  -  0.002229463219667018
apartment  -  0.3899552023941164
appliance  -  0.2710630049587336
bathroom  -  0.2182947862229208
bedroom  -  0.3525118030080429
building  -  0.3346604437837363
contact  -  0.18455420446159226
floor  -  0.34017561180833655
high  -  0.17369406775830046
kitchen  -  0.41017976140216394
room  -  0.23434499714802984
num_photos  -  0.023153773686595356
description_len  -  0.023897949088844656
mall_distance  -  0.13893007845400002
dt_distance  -  0.08225939958080011
managerID_count  -  0.052656524202606164
buildingID_count  -  0.046509195346798204
weekends  -  2.0267078877506117e-06
month_4  -  2.8758090030306382e-06
month_5  -  3.629494399907951e-05
created_nights  -  3.849636

In [62]:
for k, v in sorted(feature_rank.items(), key=itemgetter(1)):
    print (k, "  -  ", v)

weekends   -   2.0267078877506117e-06
month_4   -   2.8758090030306382e-06
created_afternoon   -   1.1796855505469966e-05
month_5   -   3.629494399907951e-05
created_morning   -   3.645831914728115e-05
created_nights   -   3.849636211319951e-05
listing_type   -   0.0007542008478153882
building_features   -   0.002229463219667018
pet_policy   -   0.002376913208605575
bedrooms   -   0.0061955250953194
unit_features   -   0.008595750400336737
bathrooms   -   0.00988099331734487
density   -   0.017704963704118636
num_photos   -   0.023153773686595356
description_len   -   0.023897949088844656
buildingID_count   -   0.046509195346798204
managerID_count   -   0.052656524202606164
longitude   -   0.060959171000756256
dt_distance   -   0.08225939958080011
price   -   0.08776029436638774
latitude   -   0.08985502052948602
building_interest   -   0.09494808536065041
manager_skill   -   0.1098756423498074
mall_distance   -   0.13893007845400002
high   -   0.17369406775830046
contact   -   0.18455

In [63]:
relevant_feat = []
from operator import itemgetter
for k, v in sorted(feature_rank.items(), key=itemgetter(1)):
    if v < 0.17:
        relevant_feat.append(k)
#         print (k, "  -  ", v)
# relevant_feat

In [64]:
target_num_map = {'high':0, 'medium':1, 'low':2}
final_train_df['int_level'] = np.array(final_train_df['interest_level'].apply(lambda x: target_num_map[x]))

In [65]:
full_X = final_train_df[relevant_feat].values
full_y = final_train_df["int_level"]

In [35]:
import xgboost as xgb

In [55]:
param4 = {
    'objective': 'multi:softprob',  
    'eta': 0.1, #
    'max_depth': 7,  #default is 6, increase this value = more complex model
    'num_class': 3,
    'eval_metric': 'mlogloss',
    'min_child_weight': 1, # default is 1, larger value = more conservative model
    'subsample': 1, # default is 1, smaller == less overfitting
    'colsample_bytree': 1, # default is 1
    'seed': 42
    }

In [66]:
dtrain = xgb.DMatrix(full_X, label=full_y)

In [67]:
steps = 500
model = xgb.train(param4, dtrain, steps)

In [68]:
test_X = final_test[relevant_feat].values
xgtest = xgb.DMatrix(test_X)

In [69]:
preds = model.predict(xgtest)

In [45]:
# labels2idx = {label: i for i, label in enumerate(clf.classes_)}

AttributeError: 'function' object has no attribute 'classes_'

In [70]:
out_df = pd.DataFrame(preds)
out_df.columns = ["high", "medium", "low"]
out_df["listing_id"] = final_test.listing_id.values
out_df.to_csv("xgb_final_withoutDESC_p5.csv", index=False)